## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Google developer API key
import sys
sys.path.append("C:/Users/laura/bootcamp/")
from api_keys import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jackson,US,42.4165,-122.8345,49.87,79,90,18.41,light rain
1,1,Campina Grande Do Sul,BR,-25.3056,-49.0553,86.90,49,1,7.72,clear sky
2,2,Luderitz,NaN,-26.6481,15.1594,63.91,78,0,18.10,clear sky
3,3,Ilulissat,GL,69.2167,-51.1000,26.62,46,75,14.97,broken clouds
4,4,Albany,US,42.6001,-73.9662,23.52,77,100,1.99,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Alugan,PH,12.2188,125.4808,77.56,79,100,15.30,overcast clouds
18,18,Vaini,TO,-21.2000,-175.2000,78.96,94,90,20.71,heavy intensity rain
20,20,Kahului,US,20.8947,-156.4700,77.85,85,20,5.75,light rain
31,31,Saint-Philippe,RE,-21.3585,55.7679,77.22,78,94,9.84,overcast clouds
41,41,Merauke,ID,-8.4667,140.3333,79.25,85,100,8.66,overcast clouds
42,42,Kismayo,SO,-0.3582,42.5454,79.09,83,12,14.81,few clouds
46,46,Mahebourg,MU,-20.4081,57.7000,79.09,78,75,11.50,broken clouds
49,49,Luwuk,ID,-0.9516,122.7875,77.11,86,100,4.21,overcast clouds
50,50,Rikitea,PF,-23.1203,-134.9692,77.16,70,100,6.53,light rain
60,60,Atuona,PF,-9.8000,-139.0333,78.64,68,23,18.68,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                99
City                   99
Country                99
Lat                    99
Lng                    99
Max Temp               99
Humidity               99
Cloudiness             99
Wind Speed             99
Current Description    99
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_new = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_new[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Alugan,PH,77.56,overcast clouds,12.2188,125.4808,
18,Vaini,TO,78.96,heavy intensity rain,-21.2000,-175.2000,
20,Kahului,US,77.85,light rain,20.8947,-156.4700,
31,Saint-Philippe,RE,77.22,overcast clouds,-21.3585,55.7679,
41,Merauke,ID,79.25,overcast clouds,-8.4667,140.3333,
42,Kismayo,SO,79.09,few clouds,-0.3582,42.5454,
46,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,
49,Luwuk,ID,77.11,overcast clouds,-0.9516,122.7875,
50,Rikitea,PF,77.16,light rain,-23.1203,-134.9692,
60,Atuona,PF,78.64,few clouds,-9.8000,-139.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for i, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat=row["Lat"]
    lng=row["Lng"]
    #add lat and lng to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 

    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #grab first hotel from the results and store the name
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")    

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Alugan,PH,77.56,overcast clouds,12.2188,125.4808,JM Lodge
18,Vaini,TO,78.96,heavy intensity rain,-21.2000,-175.2000,Keleti Beach Resort
20,Kahului,US,77.85,light rain,20.8947,-156.4700,Maui Seaside Hotel
31,Saint-Philippe,RE,77.22,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
41,Merauke,ID,79.25,overcast clouds,-8.4667,140.3333,
42,Kismayo,SO,79.09,few clouds,-0.3582,42.5454,Agan-Hotel
46,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
49,Luwuk,ID,77.11,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
50,Rikitea,PF,77.16,light rain,-23.1203,-134.9692,People ThankYou
60,Atuona,PF,78.64,few clouds,-9.8000,-139.0333,Villa Enata


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Alugan,PH,77.56,overcast clouds,12.2188,125.4808,JM Lodge
18,Vaini,TO,78.96,heavy intensity rain,-21.2000,-175.2000,Keleti Beach Resort
20,Kahului,US,77.85,light rain,20.8947,-156.4700,Maui Seaside Hotel
31,Saint-Philippe,RE,77.22,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
42,Kismayo,SO,79.09,few clouds,-0.3582,42.5454,Agan-Hotel
46,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
49,Luwuk,ID,77.11,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
50,Rikitea,PF,77.16,light rain,-23.1203,-134.9692,People ThankYou
60,Atuona,PF,78.64,few clouds,-9.8000,-139.0333,Villa Enata
68,Cidreira,BR,79.81,clear sky,-30.1811,-50.2056,Hotel Castelo


In [12]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
#add heatmap of temperature for the vacation spots

max_temp=clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.2)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))